In [1]:
#Este notebook é primordial em relação a curadoria da base
#A principal função dele é separar as variáveis examinadas em colunas, para uma janela de tempo específica
#ao fim, ele monta uma tabela só, com todas as variáveis e diagnóstico, e salvando em um arquivo .csv

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.display import display

In [2]:
path = "/scratch/haniel.botelho/physionet.org/files/eicu-crd/2.0/"


df_patient = pd.read_csv(path + 'patient.csv')
df_apacheApsVar = pd.read_csv(path + 'apacheApsVar.csv')
df_infusiondrug = pd.read_csv(path + 'infusionDrug.csv')
df_physicalExam = pd.read_csv(path + 'physicalExam.csv')
df_lab = pd.read_csv(path + 'lab.csv')
df_hospital = pd.read_csv(path + 'hospital.csv')
df_diagnosis = pd.read_csv(path + 'diagnosis.csv')
df_respiratoryCare = pd.read_csv(path + 'respiratoryCare.csv')
df_medication = pd.read_csv(path + 'medication.csv')
df_microlab = pd.read_csv(path + 'microLab.csv')

/tmp/ipykernel_252898/503048312.py:6: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_infusiondrug = pd.read_csv(path + 'infusionDrug.csv')
/tmp/ipykernel_252898/503048312.py:11: DtypeWarning: Columns (4,5,6,12,26,27,28,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_respiratoryCare = pd.read_csv(path + 'respiratoryCare.csv')
/tmp/ipykernel_252898/503048312.py:12: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_medication = pd.read_csv(path + 'medication.csv')


In [3]:
#parametrizando tempo
t0 = 0
t1 = 24*60

tempo = "alsdkf"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

In [4]:
characteristic = df_patient [['patientunitstayid', 'gender', 'age', 'ethnicity']].copy()
#Substitua valores específicos por NaN
#characteristic = characteristic.replace(-1, pd.NA)
characteristic.head()

,patientunitstayid,gender,age,ethnicity
0,141168,Female,70,Caucasian
1,141178,Female,52,Caucasian
2,141179,Female,52,Caucasian
3,141194,Male,68,Caucasian
4,141196,Male,71,Caucasian


In [4]:
# Internações: 0 por outro diagnóstico e 1 para Sepse
diagnostico = df_diagnosis [['patientunitstayid', 'diagnosisstring', 'diagnosisoffset']].copy()
diagnostico = diagnostico.query('diagnosisoffset >= 0 and diagnosisoffset <= 1440')

# Criar a coluna de diagnóstico de sepse ou não
diagnostico['sepse_diagnostico'] = 0

# Verificar se ocorre a palavra "sepse" na coluna 'apacheadmissiondx' e atribuir 1 se ocorrer
# Marcar 1 em 'sepse_diagnostico' se "sepse" ou "septic" for encontrado em 'diagnosisstring'
diagnostico.loc[diagnostico['diagnosisstring'].str.contains('sepsis|septic', case=False, na=False), 'sepse_diagnostico'] = 1

# Manter apenas as colunas de interesse
diagnostico = diagnostico[['patientunitstayid', 'sepse_diagnostico']]

# Agrupar por 'patientunitstayid' e tomar o valor máximo para indicar se teve sepse em qualquer diagnóstico nas primeiras 24h
diagnostico = diagnostico.groupby('patientunitstayid')['sepse_diagnostico'].max().reset_index()
diagnostico.query('sepse_diagnostico > 0')

,patientunitstayid,sepse_diagnostico
2,141227,1
6,141288,1
7,141289,1
9,141297,1
10,141304,1
...,...,...
170745,3352884,1
170752,3352922,1
170755,3352961,1
170793,3353145,1


In [5]:
sepse = df_patient[df_patient['apacheadmissiondx'].str.contains('sepsis',case=False,na=False)].copy()
sepse['entrada'] = 1

In [61]:
# Criar a coluna sofa_diagnostico na tabela sofa
sofa['sofa_diagnostico'] = sofa['sofascore'].map(lambda x: 1 if x >= 2 else 0)
# Agrupar por patientunitstayid e pegar o máximo de sofa_diagnostico
sofa1 = sofa.groupby('patientunitstayid')['sofa_diagnostico'].max().reset_index()
# Merge cultura e antibioticos com os dois dataframes do SOFA
infeccao = pd.merge(cultura, antibiotico, on='patientunitstayid', how='outer')

sofa1 = pd.merge(sofa1, infeccao, on='patientunitstayid', how='outer')

# Função para calcular SOFA + Suspeita de Infecção
def culturaesofa(row):
    if (row['cultura']==1 or row['antibiotico']==1) and row['sofa_diagnostico']==1:
        return 1
    else:
        return 0
    
sofa1['sc_diagnostico'] = sofa1.apply(culturaesofa, axis=1)

In [6]:
f = pd.merge(diagnostico.query('sepse_diagnostico > 0'),sepse[['patientunitstayid','entrada']],on='patientunitstayid',how='outer')
#f = pd.merge(f,sofa1.query('sc_diagnostico == 1'),on='patientunitstayid',how='outer')
f.to_csv('./eicu/septicos.csv',index=False)

In [17]:
df_patient.columns

Index(['patientunitstayid', 'patienthealthsystemstayid', 'gender', 'age',
       'ethnicity', 'hospitalid', 'wardid', 'apacheadmissiondx',
       'admissionheight', 'hospitaladmittime24', 'hospitaladmitoffset',
       'hospitaladmitsource', 'hospitaldischargeyear',
       'hospitaldischargetime24', 'hospitaldischargeoffset',
       'hospitaldischargelocation', 'hospitaldischargestatus', 'unittype',
       'unitadmittime24', 'unitadmitsource', 'unitvisitnumber', 'unitstaytype',
       'admissionweight', 'dischargeweight', 'unitdischargetime24',
       'unitdischargeoffset', 'unitdischargelocation', 'unitdischargestatus',
       'uniquepid'],
      dtype='object')

In [19]:
df_septicos = pd.read_csv('./eicu/df_septico.csv')
df_septicos.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
df_septicos = pd.merge(df_septicos,f['patientunitstayid'],on='patientunitstayid')
df_septicos = pd.merge(df_septicos,df_patient[['patientunitstayid','hospitalid']],on='patientunitstayid')
df_septicos.to_csv('./eicu/df_septico.csv',index=False)

#### sofa

##### gcs

In [46]:
tempo = "physicalexamoffset"
stringquery = f"{tempo} > {t0} and {tempo} < {t1}"

sofa = df_patient[['patientunitstayid']].copy()
df_physicalExam = df_physicalExam.query(stringquery)

gcs_filtro = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('gcs', case=False, na=False)]

# Convertendo a coluna para valores numéricos, substituindo não numéricos por NaN
gcs_filtro['physicalexamvalue'] = pd.to_numeric(gcs_filtro['physicalexamvalue'], errors='coerce')

# Removendo as linhas que contêm NaN na coluna 'physicalexampath'
gcs_filtro = gcs_filtro.dropna(subset=['physicalexamvalue'])

motor = gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Motor Score', case=False, na=False)]
motor = motor[['patientunitstayid', 'physicalexamtext','physicalexamoffset']].copy()
motor = motor.rename(columns={'physicalexamtext': 'motor'})
motor['motor'] = motor['motor'].astype(int)

#Removendo motor do df GCS
gcs_filtro = gcs_filtro.drop(gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Motor Score', case=False)].index)

eyes = gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Eyes Score', case=False, na=False)]
eyes = eyes[['patientunitstayid', 'physicalexamtext','physicalexamoffset']].copy()
eyes = eyes.rename(columns={'physicalexamtext': 'eyes'})
eyes['eyes'] = eyes['eyes'].astype(int)

#Removendo eyes do df GCS
gcs_filtro = gcs_filtro.drop(gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Eyes Score', case=False)].index)

verbal = gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Verbal Score', case=False, na=False)]
verbal = verbal[['patientunitstayid', 'physicalexamtext','physicalexamoffset']].copy()
verbal = verbal.rename(columns={'physicalexamtext': 'verbal'})
verbal['verbal'] = verbal['verbal'].astype(int)

#Removendo verbal do df GCS
gcs_filtro = gcs_filtro.drop(gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Verbal Score', case=False)].index)

gcs_total = gcs_filtro[gcs_filtro['physicalexampath'].str.contains('gcs', case=False, na=False)]
gcs_total = gcs_total[['patientunitstayid', 'physicalexamtext','physicalexamoffset']].copy()
gcs_total = gcs_total.rename(columns={'physicalexamtext': 'gcs_total'})
gcs_total['gcs_total'] = gcs_total['gcs_total'].astype(int)

#Removendo GCS do df GCS
gcs_filtro = gcs_filtro.drop(gcs_filtro[gcs_filtro['physicalexampath'].str.contains('gcs', case=False)].index)

gcs = pd.merge(motor, eyes, on=['patientunitstayid','physicalexamoffset'], how='outer')
gcs = pd.merge(gcs, verbal, on=['patientunitstayid','physicalexamoffset'], how='outer')
gcs ['gcs'] = gcs ['motor'] + gcs ['eyes'] + gcs ['verbal']

gcs.to_csv("./eicu/gcs.csv",index=False)

gcs = gcs.drop(['motor', 'eyes', 'verbal'], axis=1)
idx = gcs.groupby('patientunitstayid')['gcs'].idxmin()
idx = idx.dropna()
gcs = gcs.loc[idx]

def score(row):
  if((row['gcs']<6)):
    return 4
  elif((row['gcs']>=6) and (row['gcs']<=9)):
    return 3
  elif((row['gcs']>=10) and (row['gcs']<=12)):
    return 2
  elif((row['gcs']>=13) and (row['gcs']<=14)):
    return 1
  else:
    return 0

sofa['Nervous'] = gcs.apply(score, axis=1)
sofa


/tmp/ipykernel_246897/2198588415.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gcs_filtro['physicalexamvalue'] = pd.to_numeric(gcs_filtro['physicalexamvalue'], errors='coerce')


,patientunitstayid,Nervous
0,141168,0.0
1,141178,NaN
2,141179,1.0
3,141194,NaN
4,141196,0.0
...,...,...
200854,3353235,NaN
200855,3353237,NaN
200856,3353251,NaN
200857,3353254,NaN


##### cardio

In [47]:
#del cardio

tempo = "infusionoffset"
stringquery = f"{tempo} > {t0} and {tempo} < {t1}"

df_infusiondrug = df_infusiondrug.query(stringquery)

dopamine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Dopamine', case=False, na=False)]
dobutamine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Dobutamine', case=False, na=False)]
epinephrine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Epinephrine', case=False, na=False)]
norepinephrine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Norepinephrine', case=False, na=False)]

dopamine = dopamine[['patientunitstayid', 'drugamount', 'volumeoffluid','infusionoffset']].copy()
dobutamine = dobutamine[['patientunitstayid', 'drugamount', 'volumeoffluid','infusionoffset']].copy()
epinephrine = epinephrine[['patientunitstayid', 'drugamount', 'volumeoffluid','infusionoffset']].copy()
norepinephrine = norepinephrine[['patientunitstayid', 'drugamount', 'volumeoffluid','infusionoffset']].copy()

dopamine['concentration.dop'] = dopamine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)
epinephrine['concentration.epi'] = epinephrine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)
dobutamine['concentration.dob'] = dobutamine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)
norepinephrine['concentration.nor'] = norepinephrine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)


dopamine.rename(columns={'drugamount' : 'dopamine'}, inplace=True)
dobutamine.rename(columns={'drugamount' : 'dobutamine'}, inplace=True)
epinephrine.rename(columns={'drugamount' : 'epinephrine'}, inplace=True)
norepinephrine.rename(columns={'drugamount' : 'norepinephrine'}, inplace=True)

dopamine = dopamine.groupby('patientunitstayid')['concentration.dop'].max().reset_index()
dobutamine = dobutamine.groupby('patientunitstayid')['concentration.dob'].max().reset_index()
epinephrine = epinephrine.groupby('patientunitstayid')['concentration.epi'].max().reset_index()
norepinephrine = norepinephrine.groupby('patientunitstayid')['concentration.nor'].max().reset_index()

cardio = pd.merge(dopamine, dobutamine, on='patientunitstayid', how='outer')
cardio = pd.merge(cardio, epinephrine, on='patientunitstayid', how='outer')
cardio = pd.merge(cardio, norepinephrine, on='patientunitstayid', how='outer')

In [48]:
systolic = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('systolic', case=False, na=False)]
systolic = systolic[systolic['physicalexampath'].str.contains('current',case=False,na=False)]
diastolic = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('diastolic', case=False, na=False)]
diastolic = diastolic[diastolic['physicalexampath'].str.contains('current',case=False,na=False)]

systolic = systolic[['patientunitstayid', 'physicalexamtext','physicalexamoffset']].copy()
diastolic = diastolic[['patientunitstayid', 'physicalexamtext','physicalexamoffset']].copy()

systolic.rename(columns={'physicalexamtext' : 'systolic'}, inplace=True)
diastolic.rename(columns={'physicalexamtext' : 'diastolic'}, inplace=True)

MAP = pd.merge(systolic, diastolic, on=['patientunitstayid','physicalexamoffset'])

try:
  MAP["diastolic"] = MAP["diastolic"].astype(int)
except ValueError:
  MAP["diastolic"] = pd.to_numeric(MAP["diastolic"], errors='coerce')

try:
  MAP["systolic"] = MAP["systolic"].astype(int)
except ValueError:
  MAP["systolic"] = pd.to_numeric(MAP["systolic"], errors='coerce')

#PAM = (Pressão Sistólica + 2 x Pressão Diastólica) / 3
MAP['MAP'] = (MAP['systolic'] + 2*(MAP['diastolic'])) / 3
MAP = MAP.groupby('patientunitstayid')['MAP'].min().reset_index()

cardio = pd.merge(cardio, MAP, on='patientunitstayid', how='outer')

In [49]:

cardio.to_csv('./eicu/cardio.csv',index=False)

In [50]:
def score(row):
  if((row['concentration.dop']>15) or (row['concentration.epi']>0.1) or (row['concentration.nor']>0.1)):
    return 4
  elif((row['concentration.dop']>5) or (row['concentration.epi']<=0.1) or (row['concentration.nor']<=0.1)):
    return 3
  elif((row['concentration.dop']<=5) or (row['concentration.dop']>0)):
    return 2
  elif(row['MAP']<70):
    return 1
  else:
    return 0

sofa['Cardiovascular'] = cardio.apply(score, axis=1)

##### respiratorio

In [51]:
tempo = "labresultoffset"
stringquery = f"{tempo} >= {t0} and {tempo} <= {t1}"

df_lab = df_lab.query(stringquery)

pao2 = df_lab[df_lab['labname'].str.contains('^paO2$', case=False, na=False, regex=True)]
fio2 = df_lab[df_lab['labname'].str.contains('^FiO2$', case=False, na=False, regex=True)]

pao2 = pao2[['patientunitstayid', 'labresult','labresultoffset']].copy()
fio2 = fio2[['patientunitstayid', 'labresult','labresultoffset']].copy()

pao2.rename(columns={'labresult' : 'pao2'}, inplace=True)
fio2.rename(columns={'labresult' : 'fio2'}, inplace=True)

# Substituir 0.0 por NaN em fio2
fio2['fio2'] = fio2['fio2'].replace(0.0, float('nan'))

resp = pd.merge(pao2, fio2, on=['patientunitstayid','labresultoffset'], how='outer')
resp['Total'] = resp['pao2']/resp['fio2']

def score(row):
    if row['Total'] < 100:
        return 4
    elif 100 <= row['Total'] < 200:
        return 3
    elif 200 <= row['Total'] < 300:
        return 2
    elif 300 <= row['Total'] < 400:
        return 1
    else:
        return 0

resp = resp.groupby('patientunitstayid')['Total'].min().reset_index()
sofa['Respiratory'] = resp.apply(score, axis=1)

In [52]:
resp.to_csv('./eicu/respiratorio.csv',index=False)

##### coagulacao

In [53]:
platelets = df_lab[df_lab['labname'].str.contains('platelets', case=False, na=False)]
platelets = platelets[['patientunitstayid', 'labresult']].copy()
platelets.rename(columns={'labresult' : 'platelets'}, inplace=True)
platelets = platelets.groupby('patientunitstayid')['platelets'].min().reset_index()

def score(row):
    if row['platelets'] < 20:
        return 4
    elif 20 <= row['platelets'] < 50:
        return 3
    elif 50 <= row['platelets'] < 100:
        return 2
    elif 100 <= row['platelets'] < 150:
        return 1
    else:
        return 0

sofa['Coagulation'] = platelets.apply(score, axis=1)

In [54]:
platelets.to_csv('./eicu/coagulacao.csv',index=False)

##### liver

In [55]:
bilirubin = df_lab[df_lab['labname'].str.contains('total bilirubin', case=False, na=False)]
bilirubin = bilirubin[['patientunitstayid', 'labresult']].copy()
bilirubin.rename(columns={'labresult' : 'bilirubin'}, inplace=True)
bilirubin = bilirubin.groupby('patientunitstayid')['bilirubin'].max().reset_index()

def score(row):
    if row['bilirubin'] >= 12.0:
        return 4
    elif 12.0 > row['bilirubin'] >= 6.0:
        return 3
    elif 6.0 > row['bilirubin'] >= 2.0:
        return 2
    elif 2.0 > row['bilirubin'] > 1.2:
        return 1
    else:
        return 0

sofa['Liver'] = bilirubin.apply(score, axis=1)

In [56]:
bilirubin.to_csv('./eicu/liver.csv',index=False)

##### creatinine

In [57]:
creatinine = df_lab[df_lab['labname'].str.contains('creatinine', case=False, na=False)]
creatinine = creatinine[['patientunitstayid','labresult']].copy()
creatinine.rename(columns={'labresult' : 'creatinine'}, inplace=True)
creatinine = creatinine.groupby('patientunitstayid')['creatinine'].max().reset_index()

def score(row):
  if row['creatinine'] >= 5.0:
    return 4
  elif 5.0 > row['creatinine'] >= 3.5:
    return 3
  elif 3.0 > row['creatinine'] >= 2.0:
    return 2
  elif 2.0 > row['creatinine'] > 1.2:
    return 1
  else:
    return 0

sofa['Renal'] = creatinine.apply(score, axis=1)

In [58]:
creatinine.to_csv('./eicu/renal.csv',index=False)

##### calculo

In [59]:
sofa['sofascore'] = sofa['Nervous'] + sofa['Cardiovascular'] + sofa['Respiratory'] + sofa['Coagulation'] + sofa['Liver'] + sofa['Renal']
sofa = sofa[['patientunitstayid', 'sofascore']].copy()
sofa

,patientunitstayid,sofascore
0,141168,5.0
1,141178,NaN
2,141179,7.0
3,141194,NaN
4,141196,4.0
...,...,...
200854,3353235,NaN
200855,3353237,NaN
200856,3353251,NaN
200857,3353254,NaN


In [60]:
sofa.to_csv('./eicu/sofa.csv',index=False)

#### inflamacao

In [10]:
bands = df_lab[df_lab['labname'].str.contains('-bands', case=False, na=False)]
CRP = df_lab[df_lab['labname'].str.contains('CRP', case=False, na=False)]
ESR = df_lab[df_lab['labname'].str.contains('ESR', case=False, na=False)]
lactate = df_lab[df_lab['labname'].str.contains('lactate', case=False, na=False)]
temperature = df_lab[df_lab['labname'].str.contains('Temp', case=False, na=False)]
WBC = df_lab[df_lab['labname'].str.contains('WBC x 1000', case=False, na=False)]
lymphs = df_lab[df_lab['labname'].str.contains('-lymphs', case=False, na=False)]

bands = bands[['patientunitstayid', 'labresult']].copy()
CRP = CRP[['patientunitstayid', 'labresult']].copy()
ESR = ESR[['patientunitstayid', 'labresult']].copy()
temperature = temperature[['patientunitstayid', 'labresulttext']].copy()
lactate = lactate[['patientunitstayid', 'labresult']].copy()
WBC = WBC[['patientunitstayid', 'labresult']].copy()
lymphs = lymphs[['patientunitstayid', 'labresult']].copy()

# Transformando as temperaturas em celsius
temperature.dropna(inplace=True)
temperature['temp'] = temperature['labresulttext'].astype(float)
temperature['temp'] = temperature['temp'].map(lambda x: x if x < 60 else (x - 32) * 5/9)
# Pior valor da temperatura
temperature['distancia'] = np.abs(temperature['temp'] - 36.85)
temperature = temperature.loc[temperature.groupby('patientunitstayid')['distancia'].idxmax()]
temperature = temperature.reset_index(drop=True)
temperature = temperature.drop(['distancia', 'labresulttext'], axis=1)

# Calcular o valor mais crítico para WBC
WBC = WBC.dropna(subset=['labresult'])
WBC['distancia'] = np.abs(WBC['labresult'] - 7500)
WBC = WBC.loc[WBC.groupby('patientunitstayid')['distancia'].idxmax()]
WBC = WBC.reset_index(drop=True)
WBC = WBC.drop(['distancia'], axis=1)

# Calcular o valor mais crítico para linfócitos
lymphs = lymphs.dropna(subset=['labresult'])
lymphs['distancia'] = np.abs(lymphs['labresult'] - 2000)
lymphs = lymphs.loc[lymphs.groupby('patientunitstayid')['distancia'].idxmax()]
lymphs = lymphs.reset_index(drop=True)
lymphs = lymphs.drop(['distancia'], axis=1)

bands = bands.groupby('patientunitstayid')['labresult'].max().reset_index()
CRP = CRP.groupby('patientunitstayid')['labresult'].max().reset_index()
ESR = ESR.groupby('patientunitstayid')['labresult'].max().reset_index()
lactate= lactate.groupby('patientunitstayid')['labresult'].max().reset_index()

bands = bands.rename(columns={'labresult': 'bands'})
CRP = CRP.rename(columns={'labresult': 'CRP'})
ESR = ESR.rename(columns={'labresult': 'ESR'})
lactate = lactate.rename(columns={'labresult': 'lactate'})
WBC = WBC.rename(columns={'labresult': 'WBC'})
lymphs = lymphs.rename(columns={'labresult': 'lymphs'})

inflammation = pd.merge(bands, CRP, on='patientunitstayid', how='outer')
inflammation = pd.merge(inflammation, ESR, on='patientunitstayid', how='outer')
inflammation = pd.merge(inflammation, temperature, on='patientunitstayid', how='outer')
inflammation = pd.merge(inflammation, lactate, on='patientunitstayid', how='outer')
inflammation = pd.merge(inflammation, WBC, on='patientunitstayid', how='outer')
inflammation = pd.merge(inflammation, lymphs, on='patientunitstayid', how='outer')

inflammation

,patientunitstayid,bands,CRP,ESR,temp,lactate,WBC,lymphs
0,141168,NaN,NaN,NaN,NaN,12.2,9.8,6.0
1,141178,NaN,NaN,NaN,NaN,NaN,7.6,45.0
2,141179,NaN,NaN,NaN,NaN,NaN,8.1,NaN
3,141194,NaN,NaN,NaN,NaN,1.9,4.3,1.0
4,141196,NaN,NaN,NaN,NaN,0.8,12.7,3.0
...,...,...,...,...,...,...,...,...
189853,3353235,NaN,NaN,NaN,NaN,NaN,5.4,13.0
189854,3353237,NaN,NaN,NaN,NaN,NaN,5.7,16.0
189855,3353251,NaN,NaN,NaN,NaN,9.8,11.9,4.0
189856,3353254,NaN,NaN,NaN,NaN,1.3,9.8,7.0


#### pulmonary

In [11]:
respE = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('Resp Current', case=False, na=False)]
respE = respE[['patientunitstayid', 'physicalexamtext','physicalexamoffset']].copy()
# Converter a coluna para numérica com valores não numéricos como NaN
respE['physicalexamtext'] = pd.to_numeric(respE['physicalexamtext'], errors='coerce')

# Adiciona a coluna 'distancia'
respE['distancia'] = np.abs(respE['physicalexamtext'] - 16)

# Seleciona os registros com a maior 'distancia' por 'patientunitstayid'
respE = respE.loc[respE.groupby('patientunitstayid')['distancia'].idxmax()]

# Reseta o índice e remove a coluna 'distancia' se não for mais necessária
respE = respE.reset_index(drop=True).drop(columns=['distancia'])

respE = respE.groupby('patientunitstayid')['physicalexamtext'].max().reset_index()
respE = respE.rename(columns={'physicalexamtext': 'resp'})

In [ ]:
respE

In [13]:
spo2 = df_lab[df_lab['labname'].str.contains('O2 Sat', case=False, na=False)]
fio2 = df_lab[df_lab['labname'].str.contains('FiO2', case=False, na=False)]

spo2 = spo2[['patientunitstayid', 'labresult', 'labresultoffset']].copy()
fio2 = fio2[['patientunitstayid', 'labresult', 'labresultoffset']].copy()

spo2 = spo2.rename(columns={'labresult': 'spo2'})
fio2 = fio2.rename(columns={'labresult': 'fio2'})

# Substituir 0.0 por NaN em fio2
fio2['fio2'] = fio2['fio2'].replace(0.0, float('nan'))

spo2_fio2 = pd.merge(spo2, fio2, on=['patientunitstayid','labresultoffset'])
spo2_fio2['spo2/fio2'] = spo2_fio2['spo2'] / spo2_fio2['fio2']
spo2_fio2 = spo2_fio2.drop(['spo2', 'fio2'], axis=1)
spo2_fio2 = spo2_fio2.groupby('patientunitstayid')['spo2/fio2'].max().reset_index()

In [14]:
pao2 = df_lab[df_lab['labname'].str.contains('paO2', case=False, na=False)]
pao2 = pao2[['patientunitstayid', 'labresult','labresultoffset']].copy()
pao2 = pao2.rename(columns={'labresult': 'pao2'})

# Substituir 0.0 por NaN em fio2
fio2['fio2'] = fio2['fio2'].replace(0.0, float('nan'))

pao2_fio2 = pd.merge(pao2, fio2, on=['patientunitstayid','labresultoffset'])

pao2_fio2['pao2/fio2'] = pao2_fio2['pao2'] / pao2_fio2['fio2']
pao2_fio2 = pao2_fio2.drop(['pao2', 'fio2'], axis=1)
pao2_fio2 = pao2_fio2.groupby('patientunitstayid')['pao2/fio2'].min().reset_index()

In [15]:
pulmonary = pd.merge(respE, spo2_fio2, on='patientunitstayid', how='outer')
pulmonary = pd.merge(pulmonary, pao2_fio2, on='patientunitstayid', how='outer')
pulmonary

,patientunitstayid,resp,spo2/fio2,pao2/fio2
0,141168,NaN,NaN,0.420000
1,141194,25.0,NaN,3.571429
2,141197,24.0,NaN,NaN
3,141203,NaN,NaN,0.510000
4,141227,NaN,NaN,1.170000
...,...,...,...,...
142133,3353226,14.0,100.000000,1.800000
142134,3353237,29.0,NaN,NaN
142135,3353251,24.0,213.333333,0.800000
142136,3353254,24.0,NaN,NaN


#### cardiovascular

In [16]:
bicarbonate = df_lab[df_lab['labname'].str.contains('bicarbonate', case=False, na=False)]
bicarbonate = bicarbonate[['patientunitstayid', 'labresult']].copy()
bicarbonate.rename(columns={'labresult': 'bicarbonate'}, inplace=True)
bicarbonate = bicarbonate.groupby('patientunitstayid')['bicarbonate'].min().reset_index()

In [17]:
heartRate = df_physicalExam[df_physicalExam['physicalexamvalue'].str.contains('HR Current', case=False, na=False)]
heartRate = heartRate[['patientunitstayid', 'physicalexamtext']].copy()

heartRate['physicalexamtext'] = heartRate['physicalexamtext'].astype(float)
heartRate['distancia'] = np.abs(heartRate['physicalexamtext'] - 75)
heartRate = heartRate.loc[heartRate.groupby('patientunitstayid')['distancia'].idxmax()]
heartRate = heartRate.reset_index(drop=True)
heartRate = heartRate.drop(['distancia'], axis=1)

heartRate.rename(columns={'physicalexamtext': 'heartRate'}, inplace=True)

In [19]:
systolic = df_physicalExam[df_physicalExam['physicalexamvalue'].str.contains(r'systolic.*Current|Current.*systolic', case=False, na=False)]
systolic = systolic[['patientunitstayid', 'physicalexamtext','physicalexamoffset']].copy()
systolic['physicalexamtext'] = systolic['physicalexamtext'].map(lambda x: pd.NA if not x.isnumeric() else float(x))
systolic.dropna(inplace=True)
systolic.rename(columns={'physicalexamtext': 'systolic'}, inplace=True)

diastolic = df_physicalExam[df_physicalExam['physicalexamvalue'].str.contains(r'diastolic.*Current|Current.*diastolic', case=False, na=False)]
diastolic = diastolic[['patientunitstayid', 'physicalexamtext','physicalexamoffset']].copy()
diastolic['physicalexamtext'] = diastolic['physicalexamtext'].map(lambda x: pd.NA if not x.isnumeric() else float(x))
diastolic.dropna(inplace=True)
diastolic.rename(columns={'physicalexamtext': 'diastolic'}, inplace=True)

MAP = pd.merge(systolic[['patientunitstayid', 'systolic','physicalexamoffset']],
              diastolic[['patientunitstayid', 'diastolic','physicalexamoffset']],
              on=['patientunitstayid','physicalexamoffset'])

try:
  MAP["diastolic"] = MAP["diastolic"].astype(int)
except ValueError:
  MAP["diastolic"] = pd.to_numeric(MAP["diastolic"], errors='coerce')

try:
  MAP["systolic"] = MAP["systolic"].astype(int)
except ValueError:
  MAP["systolic"] = pd.to_numeric(MAP["systolic"], errors='coerce')

MAP['MAP'] = (MAP['systolic'] + 2*(MAP['diastolic'])) / 3
MAP = MAP.groupby('patientunitstayid')['MAP'].min().reset_index()

systolic['distancia'] = np.abs(systolic['systolic'] - 115)
systolic = systolic.loc[systolic.groupby('patientunitstayid')['distancia'].idxmax()]
systolic = systolic.drop_duplicates(subset='patientunitstayid')
systolic = systolic.reset_index(drop=True)
systolic = systolic.drop(['distancia','physicalexamoffset'], axis=1)

diastolic['distancia'] = np.abs(diastolic['diastolic'] - 62.5)
diastolic = diastolic.loc[diastolic.groupby('patientunitstayid')['distancia'].idxmax()]
diastolic = diastolic.drop_duplicates(subset='patientunitstayid')
diastolic = diastolic.reset_index(drop=True)
diastolic = diastolic.drop(['distancia','physicalexamoffset'], axis=1)

In [20]:
troponin = df_lab[df_lab['labname'].str.contains('troponin', case=False, na=False)]
troponin = troponin[['patientunitstayid', 'labresult']].copy()
troponin.rename(columns={'labresult': 'troponin'}, inplace=True)
troponin = troponin.groupby('patientunitstayid')['troponin'].max().reset_index()

In [21]:
dfs = [
    ('bicarbonate', bicarbonate),
    ('heartRate', heartRate),
    ('troponin', troponin),
    ('systolic', systolic),
    ('diastolic', diastolic),
    ('MAP', MAP)
]

# Exibir colunas de cada DataFrame
for name, df in dfs:
    print(f"Colunas do DataFrame {name}:")
    print(df.columns.tolist())
    print()  # Linha em branco para melhor visualização

Colunas do DataFrame bicarbonate:
['patientunitstayid', 'bicarbonate']

Colunas do DataFrame heartRate:
['patientunitstayid', 'heartRate']

Colunas do DataFrame troponin:
['patientunitstayid', 'troponin']

Colunas do DataFrame systolic:
['patientunitstayid', 'systolic']

Colunas do DataFrame diastolic:
['patientunitstayid', 'diastolic']

Colunas do DataFrame MAP:
['patientunitstayid', 'MAP']



In [22]:
cardiovascular = pd.merge(bicarbonate, heartRate, on='patientunitstayid', how='outer')
cardiovascular = pd.merge(cardiovascular, troponin, on='patientunitstayid', how='outer')
cardiovascular = pd.merge(cardiovascular, systolic, on='patientunitstayid', how='outer')
cardiovascular = pd.merge(cardiovascular, diastolic, on='patientunitstayid', how='outer')
cardiovascular = pd.merge(cardiovascular, MAP, on='patientunitstayid', how='outer')
cardiovascular

,patientunitstayid,bicarbonate,heartRate,troponin,systolic,diastolic,MAP
0,141168,18.0,NaN,0.26,NaN,NaN,NaN
1,141178,25.0,NaN,NaN,NaN,NaN,NaN
2,141179,22.0,NaN,NaN,NaN,NaN,NaN
3,141194,12.0,112.0,NaN,84.0,34.0,50.666667
4,141196,30.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
192866,3353235,24.0,NaN,0.04,161.0,98.0,119.000000
192867,3353237,27.0,91.0,0.19,144.0,80.0,91.333333
192868,3353251,19.0,90.0,9.38,107.0,54.0,71.666667
192869,3353254,20.0,76.0,0.17,80.0,50.0,60.000000


#### renal

In [23]:
BUN = df_lab[df_lab['labname'].str.contains('BUN', case=False, na=False)]
BUN = BUN[['patientunitstayid', 'labresult']].copy()
BUN.rename(columns={'labresult' : 'bloodUreaNitrogen'}, inplace=True)
BUN = BUN.groupby('patientunitstayid')['bloodUreaNitrogen'].max().reset_index()
renal = BUN

renal

,patientunitstayid,bloodUreaNitrogen
0,141168,29.0
1,141178,11.0
2,141179,21.0
3,141194,41.0
4,141196,24.0
...,...,...
190210,3353235,24.0
190211,3353237,32.0
190212,3353251,91.0
190213,3353254,45.0


#### hepatic

In [24]:
# Selecionar indicadores hepáticos no dataset 'lab'
alanine = df_lab[df_lab['labname'].str.contains('ALT', case=False, na=False)]
aspartate = df_lab[df_lab['labname'].str.contains('AST', case=False, na=False)]
bilirubin = df_lab[df_lab['labname'].str.contains('total bilirubin', case=False, na=False)]

# Fazer uma cópia com as colunas importantes para o modelo
alanine = alanine[['patientunitstayid', 'labresult']].copy()
aspartate = aspartate[['patientunitstayid', 'labresult']].copy()
bilirubin = bilirubin[['patientunitstayid', 'labresult']].copy()

# Renomear colunas para facilitar visualização
alanine.rename(columns={'labresult' : 'alanine'}, inplace=True)
aspartate.rename(columns={'labresult' : 'aspartate'}, inplace=True)
bilirubin.rename(columns={'labresult' : 'bilirubin'}, inplace=True)

# Filtrar os valores mais críticos de cada paciente
alanine = alanine.groupby('patientunitstayid')['alanine'].max().reset_index()
aspartate = aspartate.groupby('patientunitstayid')['aspartate'].max().reset_index()
bilirubin = bilirubin.groupby('patientunitstayid')['bilirubin'].max().reset_index()

# Mesclar os datasets
hepatic = pd.merge(alanine, aspartate, on='patientunitstayid', how='outer')
hepatic = pd.merge(hepatic, bilirubin, on='patientunitstayid', how='outer')

hepatic

,patientunitstayid,alanine,aspartate,bilirubin
0,141168,1189.0,4257.0,5.2
1,141178,52.0,40.0,0.4
2,141194,20.0,24.0,0.4
3,141196,18.0,15.0,0.3
4,141197,19.0,17.0,0.3
...,...,...,...,...
140792,3353216,9.0,20.0,0.2
140793,3353226,36.0,31.0,0.4
140794,3353237,12.0,18.0,0.7
140795,3353251,272.0,264.0,0.5


#### hematologic

In [25]:
hemoglobin = df_lab[df_lab['labname'].str.contains('Hgb', case=False, na=False)]
hemoglobin = hemoglobin[['patientunitstayid', 'labresult']].copy()
hemoglobin.rename(columns={'labresult' : 'hemoglobin'}, inplace=True)
hemoglobin = hemoglobin.groupby('patientunitstayid')['hemoglobin'].min().reset_index()

In [26]:
iqr = df_lab[df_lab['labname'].str.contains('INR', case=False, na=False)]
iqr = iqr[['patientunitstayid', 'labresult']].copy()
iqr.rename(columns={'labresult' : 'iqr'}, inplace=True)
iqr = iqr.groupby('patientunitstayid')['iqr'].max().reset_index()

In [27]:
#ja foi
platelets = df_lab[df_lab['labname'].str.contains('platelets', case=False, na=False)]
platelets = platelets[['patientunitstayid', 'labresult']].copy()
platelets.rename(columns={'labresult' : 'platelets'}, inplace=True)
platelets = platelets.groupby('patientunitstayid')['platelets'].min().reset_index()

In [28]:
PTT = df_lab[df_lab['labname'].str.contains('PTT', case=False, na=False)]
PTT = PTT[['patientunitstayid', 'labresult']].copy()
PTT.rename(columns={'labresult' : 'PTT'}, inplace=True)
PTT = PTT.groupby('patientunitstayid')['PTT'].max().reset_index()

In [29]:
fibrinogen = df_lab[df_lab['labname'].str.contains('fibrinogen', case=False, na=False)]
fibrinogen = fibrinogen[['patientunitstayid', 'labresult']].copy()

fibrinogen['distancia'] = np.abs(fibrinogen['labresult'] - 300)
fibrinogen = fibrinogen.dropna(subset=['distancia'])
fibrinogen = fibrinogen.loc[fibrinogen.groupby('patientunitstayid')['distancia'].idxmax()]
fibrinogen = fibrinogen.reset_index(drop=True)
fibrinogen = fibrinogen.drop(['distancia'], axis=1)

fibrinogen.rename(columns={'labresult' : 'fibrinogen'}, inplace=True)

In [30]:
hematologic = pd.merge(hemoglobin, iqr, on='patientunitstayid', how='outer')
hematologic = pd.merge(hematologic, platelets, on='patientunitstayid', how='outer')
hematologic = pd.merge(hematologic, PTT, on='patientunitstayid', how='outer')
hematologic = pd.merge(hematologic, fibrinogen, on='patientunitstayid', how='outer')
hematologic

,patientunitstayid,hemoglobin,iqr,platelets,PTT,fibrinogen
0,141168,11.4,4.20,187.0,36.0,177.0
1,141178,15.5,NaN,273.0,NaN,NaN
2,141179,12.5,NaN,219.0,NaN,NaN
3,141194,7.7,1.10,125.0,39.0,NaN
4,141196,10.3,NaN,453.0,NaN,NaN
...,...,...,...,...,...,...
190100,3353235,16.3,NaN,133.0,NaN,NaN
190101,3353237,11.6,2.39,128.0,114.0,NaN
190102,3353251,8.7,1.15,106.0,29.0,NaN
190103,3353254,7.3,1.03,132.0,26.0,NaN


#### bioquimica

In [31]:
albumin = df_lab[df_lab['labname'].str.contains('albumin', case=False, na=False)]
albumin = albumin[['patientunitstayid', 'labresult']].copy()
albumin.rename(columns={'labresult' : 'albumin'}, inplace=True)
albumin = albumin.groupby('patientunitstayid')['albumin'].min().reset_index()

In [32]:
chloride = df_lab[df_lab['labname'].str.contains('chloride', case=False, na=False)]
chloride = chloride[['patientunitstayid', 'labresult']].copy()

chloride['distancia'] = np.abs(chloride['labresult'] - 102.5)
chloride = chloride.dropna(subset=['distancia'])
chloride = chloride.loc[chloride.groupby('patientunitstayid')['distancia'].idxmax()]
chloride = chloride.reset_index(drop=True)
chloride = chloride.drop(['distancia'], axis=1)

chloride.rename(columns={'labresult' : 'chloride'}, inplace=True)

In [33]:
glucose = df_lab[df_lab['labname'].str.contains('glucose', case=False, na=False)]
glucose = glucose[['patientunitstayid', 'labresult']].copy()

glucose['distancia'] = np.abs(glucose['labresult'] - 125)
glucose = glucose.dropna(subset=['distancia'])
glucose = glucose.loc[glucose.groupby('patientunitstayid')['distancia'].idxmax()]
glucose = glucose.reset_index(drop=True)
glucose = glucose.drop(['distancia'], axis=1)

glucose.rename(columns={'labresult' : 'glucose'}, inplace=True)

In [34]:
sodium = df_lab[df_lab['labname'].str.contains('sodium', case=False, na=False)]
sodium = sodium[['patientunitstayid', 'labresult']].copy()

sodium['distancia'] = np.abs(sodium['labresult'] - 140)
sodium = sodium.dropna(subset=['distancia'])
sodium = sodium.loc[sodium.groupby('patientunitstayid')['distancia'].idxmax()]
sodium = sodium.reset_index(drop=True)
sodium = sodium.drop(['distancia'], axis=1)

sodium.rename(columns={'labresult' : 'sodium'}, inplace=True)

In [35]:
potassium = df_lab[df_lab['labname'].str.contains('potassium', case=False, na=False)]
potassium = potassium[['patientunitstayid', 'labresult']].copy()

potassium['distancia'] = np.abs(potassium['labresult'] - 4.5)
potassium = potassium.dropna(subset=['distancia'])
potassium = potassium.loc[potassium.groupby('patientunitstayid')['distancia'].idxmax()]
potassium = potassium.reset_index(drop=True)
potassium = potassium.drop(['distancia'], axis=1)

potassium.rename(columns={'labresult' : 'potassium'}, inplace=True)

In [36]:
calcium = df_lab[df_lab['labname'].str.contains('calcium', case=False, na=False)]
calcium = calcium[['patientunitstayid', 'labresult']].copy()

calcium['distancia'] = np.abs(calcium['labresult'] - 9.6)
calcium = calcium.dropna(subset=['distancia'])
calcium = calcium.loc[calcium.groupby('patientunitstayid')['distancia'].idxmax()]
calcium = calcium.reset_index(drop=True)
calcium = calcium.drop(['distancia'], axis=1)

calcium.rename(columns={'labresult' : 'calcium'}, inplace=True)

In [37]:
phosphate = df_lab[df_lab['labname'].str.contains('phosphate', case=False, na=False)]
phosphate = phosphate[['patientunitstayid', 'labresult']].copy()

phosphate['distancia'] = np.abs(phosphate['labresult'] - 3.5)
phosphate = phosphate.dropna(subset=['distancia'])
phosphate = phosphate.loc[phosphate.groupby('patientunitstayid')['distancia'].idxmax()]
phosphate = phosphate.reset_index(drop=True)
phosphate = phosphate.drop(['distancia'], axis=1)

phosphate.rename(columns={'labresult' : 'phosphate'}, inplace=True)

In [38]:
magnesium = df_lab[df_lab['labname'].str.contains('magnesium', case=False, na=False)]
magnesium = magnesium[['patientunitstayid', 'labresult']].copy()
magnesium.rename(columns={'labresult' : 'magnesium'}, inplace=True)
magnesium = magnesium.groupby('patientunitstayid')['magnesium'].min().reset_index()

In [39]:
bioquimica = pd.merge(albumin, chloride, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, glucose, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, sodium, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, gcs, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, potassium, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, calcium, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, phosphate, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, magnesium, on='patientunitstayid', how='outer')
bioquimica

,patientunitstayid,albumin,chloride,glucose,sodium,physicalexamoffset,gcs,potassium,calcium,phosphate,magnesium
0,141168,3.0,101.0,54.0,12.0,60.0,15.0,4.0,8.5,7.9,2.0
1,141178,4.0,108.0,77.0,146.0,NaN,NaN,3.6,8.0,NaN,NaN
2,141179,NaN,109.0,54.0,146.0,NaN,NaN,3.6,7.8,NaN,1.7
3,141194,2.3,109.0,224.0,134.0,8.0,13.0,3.2,6.9,NaN,1.2
4,141196,2.5,97.0,144.0,135.0,NaN,NaN,4.1,8.7,NaN,2.4
...,...,...,...,...,...,...,...,...,...,...,...
196375,3353235,NaN,109.0,100.0,137.0,11.0,15.0,4.1,9.8,NaN,2.1
196376,3353237,3.5,98.0,91.0,137.0,131.0,14.0,3.3,8.5,NaN,2.0
196377,3353251,3.0,99.0,346.0,67.0,78.0,3.0,3.1,3.8,7.3,1.4
196378,3353254,3.9,117.0,181.0,144.0,31.0,15.0,5.7,4.5,3.7,1.4


#### medicamento

In [40]:
tempo = "drugorderoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"
df_medication = df_medication.query(querypred)

antibiotico = df_medication [['patientunitstayid', 'drugorderoffset', 'drugname', 'dosage']].copy()

# Lista de antibióticos
antibioticos_list = [
    'Vancomycin', 'Metronidazole', 'Ceftriaxone', 'Cefazolin', 'Piperacillin-tazobactam',
    'Azithromycin', 'Cefepime', 'Penicillin', 'Clindamicina', 'Levofloxacino',
    'Meropenem', 'Levofloxacin', 'Ciprofloxacin', 'Zosyn'
]
regex_pattern = '|'.join(antibioticos_list)
antibiotico['drugname'] = antibiotico['drugname'].fillna('0')
# Criando a coluna 'antibiotico' com 1 se o paciente usou algum dos antibióticos e 0 caso contrário
antibiotico['antibiotico'] = antibiotico['drugname'].str.contains(regex_pattern, case=False, regex=True).astype(int)
antibiotico = antibiotico.loc[antibiotico.groupby('patientunitstayid')['antibiotico'].idxmax()]
antibiotico = antibiotico.reset_index(drop=True)
antibiotico = antibiotico.drop(['drugorderoffset', 'drugname' ,	'dosage'], axis=1)

#### cultura

In [41]:
tempo = "culturetakenoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"
df_microlab = df_microlab.query(querypred)

cultura = df_microlab [['patientunitstayid', 'culturetakenoffset', 'culturesite', 'organism']].copy()

cultura = cultura.drop_duplicates(subset='patientunitstayid')
cultura['cultura'] = 1
cultura = cultura.drop(['culturetakenoffset', 'culturesite' ,	'organism'], axis=1)

#### merge

In [69]:
#bioquimica.drop(['physicalexamoffset'],axis=1,inplace=True)
bioquimica.drop(['physicalexamoffset'],axis=1,inplace=True)

In [71]:
df_final = pd.merge(characteristic, inflammation, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, pulmonary, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, cardiovascular, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, renal, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, hepatic, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, hematologic, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, bioquimica, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, sofa, on='patientunitstayid', how='outer')
df_final.head()

,patientunitstayid,gender,age,ethnicity,bands,CRP,ESR,temp,lactate,WBC,...,chloride,glucose,sodium,gcs,potassium,calcium,phosphate,magnesium,sofascore,sofa_diagnostico
0,141168,Female,70,Caucasian,NaN,NaN,NaN,NaN,12.2,9.8,...,101.0,54.0,12.0,15.0,4.0,8.5,7.9,2.0,5.0,1
1,141178,Female,52,Caucasian,NaN,NaN,NaN,NaN,NaN,7.6,...,108.0,77.0,146.0,NaN,3.6,8.0,NaN,NaN,NaN,0
2,141179,Female,52,Caucasian,NaN,NaN,NaN,NaN,NaN,8.1,...,109.0,54.0,146.0,NaN,3.6,7.8,NaN,1.7,7.0,1
3,141194,Male,68,Caucasian,NaN,NaN,NaN,NaN,1.9,4.3,...,109.0,224.0,134.0,13.0,3.2,6.9,NaN,1.2,NaN,0
4,141196,Male,71,Caucasian,NaN,NaN,NaN,NaN,0.8,12.7,...,97.0,144.0,135.0,NaN,4.1,8.7,NaN,2.4,4.0,1


In [73]:
df_final.columns
df_final.drop(['sofa_diagnostico'],axis=1,inplace=True)

In [74]:
df_final.to_csv('./eicu/df_final.csv')

In [75]:
df_septico = pd.merge(df_final,f['patientunitstayid'],on='patientunitstayid')
df_septico

,patientunitstayid,gender,age,ethnicity,bands,CRP,ESR,temp,lactate,WBC,...,albumin,chloride,glucose,sodium,gcs,potassium,calcium,phosphate,magnesium,sofascore
0,141194,Male,68,Caucasian,NaN,NaN,NaN,NaN,1.9,4.3,...,2.3,109.0,224.0,134.0,13.0,3.2,6.9,NaN,1.2,NaN
1,141196,Male,71,Caucasian,NaN,NaN,NaN,NaN,0.8,12.7,...,2.5,97.0,144.0,135.0,NaN,4.1,8.7,NaN,2.4,4.0
2,141197,Male,71,Caucasian,NaN,NaN,NaN,NaN,NaN,29.5,...,2.9,99.0,162.0,139.0,15.0,4.1,9.0,NaN,NaN,9.0
3,141203,Female,77,Caucasian,NaN,NaN,NaN,NaN,3.5,8.5,...,1.4,110.0,95.0,145.0,5.0,3.6,8.0,NaN,1.5,8.0
4,141227,Male,82,Caucasian,10.0,NaN,NaN,NaN,4.3,42.7,...,NaN,101.0,157.0,132.0,13.0,3.9,8.3,NaN,1.9,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40862,3352922,Male,77,Caucasian,NaN,NaN,NaN,NaN,7.0,3.8,...,3.4,113.0,215.0,150.0,7.0,3.2,4.6,2.2,1.7,NaN
40863,3352940,Female,23,Caucasian,NaN,NaN,NaN,NaN,4.0,5.7,...,2.5,88.0,323.0,33.0,9.0,3.9,7.5,5.1,1.5,NaN
40864,3352961,Male,70,Caucasian,NaN,NaN,NaN,NaN,1.3,9.4,...,2.6,106.0,198.0,134.0,8.0,3.5,7.8,2.7,1.5,NaN
40865,3353145,Male,51,African American,NaN,NaN,NaN,NaN,NaN,28.4,...,2.6,99.0,273.0,143.0,13.0,2.6,8.8,2.5,1.5,NaN


In [76]:
df_septico.to_csv('./eicu/df_septico.csv')

In [ ]:
dfs = [
    ('characteristic', characteristic),
    ('inflammation', inflammation),
    ('pulmonary', pulmonary),
    ('cardiovascular', cardiovascular),
    ('renal', renal),
    ('hepatic', hepatic),
    ('hematologic', hematologic),
    ('bioquimica', bioquimica),
    ('gcs', gcs),
    ('sofa', sofa)
]

# Itere sobre cada DataFrame e exiba suas colunas
for name, df in dfs:
    print(f"Colunas de {name}:")
    print(df.columns.tolist())
    print()